In [13]:
import pandas as pd
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import numpy as np

In [3]:
data = pd.read_csv('test.csv')

In [4]:
data.head()

,id,tweet
0,31963,#studiolife #aislife #requires #passion #dedic...
1,31964,@user #white #supremacists want everyone to s...
2,31965,safe ways to heal your #acne!! #altwaystohe...
3,31966,is the hp and the cursed child book up for res...
4,31967,"3rd #bihday to my amazing, hilarious #nephew..."


In [5]:
data = data['tweet']

In [6]:
data.head()

0    #studiolife #aislife #requires #passion #dedic...
1     @user #white #supremacists want everyone to s...
2    safe ways to heal your #acne!!    #altwaystohe...
3    is the hp and the cursed child book up for res...
4      3rd #bihday to my amazing, hilarious #nephew...
Name: tweet, dtype: object

In [8]:
ps = PorterStemmer()
corpus = []
for i in range(0, len(data)):
    print(i,end = '\r')
    review = re.sub('[^a-zA-Z]', ' ', data[i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [9]:
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [11]:
voc_size = 500000
onehot_repr=[one_hot(word,voc_size)for word in corpus]
sent_length=25
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[     0      0      0 ... 206424 482606 344323]
 [     0      0      0 ... 443281 205966 480341]
 [     0      0      0 ... 498181 495672  20568]
 ...
 [     0      0      0 ... 301131  72865  81416]
 [     0      0      0 ... 365164 432535 186423]
 [     0      0      0 ... 192771 434973 168290]]


In [14]:
X_final=np.array(embedded_docs)

In [17]:
import keras

In [19]:
loaded_model = keras.models.load_model('finalized_model.h5')

In [20]:
loaded_model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 25, 64)            32000000  
                                                                 
 dropout_6 (Dropout)         (None, 25, 64)            0         
                                                                 
 lstm_4 (LSTM)               (None, 25, 100)           66000     
                                                                 
 dropout_7 (Dropout)         (None, 25, 100)           0         
                                                                 
 lstm_5 (LSTM)               (None, 256)               365568    
                                                                 
 dropout_8 (Dropout)         (None, 256)               0         
                                                                 
 dense_2 (Dense)             (None, 1)                

In [48]:
y_pred=loaded_model.predict(X_final)

538/538 [==============================] - 3s 6ms/step


In [69]:
y_p = [1 if x>0.9 else 0 for x in y_pred]

In [70]:
Y = pd.DataFrame(y_p, columns = ['Predicted Value'])

In [71]:
output = pd.DataFrame(data.values,columns = ['Tweet'])

In [72]:
output = output.join(Y)

In [73]:
output.head()

,Tweet,Predicted Value
0,#studiolife #aislife #requires #passion #dedic...,0
1,@user #white #supremacists want everyone to s...,0
2,safe ways to heal your #acne!! #altwaystohe...,0
3,is the hp and the cursed child book up for res...,1
4,"3rd #bihday to my amazing, hilarious #nephew...",0


In [74]:
output.to_csv('Test_data_outputs.csv')